In [5]:
# PostgreSQL 9.5
# with PostGIS 2.2
# 1. create user with credentials in db_settings.py
# 2. run query below
"""
CREATE DATABASE wells
    WITH 
    OWNER = wellsuser
    ENCODING = 'UTF8'
    CONNECTION LIMIT = -1;
GRANT ALL ON DATABASE wells TO wellsuser;
\connect wells
CREATE EXTENSION postgis;
"""

import db_settings
import psycopg2

def run_query(query):
    # Connect to the database
    conn = psycopg2.connect(
            database=db_settings.DB,
            user=db_settings.USER,
            password=db_settings.PASSWD,
            host=db_settings.HOST)
    with conn:
        with conn.cursor() as curs:
            curs.execute(query)
            print curs.statusmessage
    conn.close()
    

In [9]:
query = """
DROP TABLE IF EXISTS states CASCADE;
CREATE TABLE states 
(
    id serial,
    state char(2) UNIQUE,
    name VARCHAR,
    source_url VARCHAR,
    description TEXT,
    last_updated TIMESTAMPTZ,
    api_prefix CHAR(2),
    PRIMARY KEY (id)
);
CREATE INDEX idx_state_id ON states(id);
CREATE INDEX idx_state_st ON states(state);
"""
run_query(query)


CREATE INDEX


In [10]:
# Injection Wells: https://www.epa.gov/uic/class-ii-oil-and-gas-related-injection-wells
# Well types and statuses: http://www.ogsrlibrary.com/definitions_well_status_oil_gas_ontario  
# INJECTION = FRACKING
query = """
DROP TYPE IF EXISTS enum_well_type CASCADE;
CREATE TYPE enum_well_type AS ENUM
(
    'OIL',
    'GAS',
    'OILANDGAS',
    'INJECTION',
    'STORAGE',
    'DISPOSAL',
    'DRY HOLE',
    'OBSERVATION',
    'TEST',
    'WATER',
    'OTHER'
);
DROP TYPE IF EXISTS enum_well_status CASCADE;
CREATE TYPE enum_well_status AS ENUM
(
    'UNKNOWN',
    'PERMITTED',
    'CANCELLED',
    'ACTIVE',
    'TA',
    'PA',
    'ORPHAN'
);
DROP TYPE IF EXISTS enum_well_category CASCADE;
CREATE TYPE enum_well_category AS ENUM
(
    'CONVENTIONAL',
    'UNCONVENTIONAL',
    'FRAC',
    'CBM',
    'UIC',
    'TEST',
    'STORAGE',
    'WATER',
    'OTHER'
);
DROP TYPE IF EXISTS enum_location_src CASCADE;
CREATE TYPE enum_location_src AS ENUM
(
    'S', --Surface
    'BH' --Bottom Hole
);

DROP TABLE IF EXISTS wells;
CREATE TABLE wells
(
    id SERIAL PRIMARY KEY,
    location GEOGRAPHY(POINT, 4326) NOT NULL,
    location_src enum_location_src,
    api varchar NOT NULL CONSTRAINT plain_api CHECK (api ~ '^[A-Z0-9]+$'),
    state char(2) REFERENCES states(state) NOT NULL,
    date DATE,
    status enum_well_status NOT NULL,
    type enum_well_type,
    category enum_well_category,
    capture_time TIMESTAMPTZ,
    description TEXT,
    UNIQUE (api, status, date)
);

CREATE INDEX idx_well_id ON wells(id);
CREATE INDEX idx_api ON wells(api);
CREATE INDEX idx_api_status_date ON wells(api, status, date);
CREATE INDEX idx_api_capture_time ON wells(api, capture_time);
CREATE INDEX idx_well_location ON wells(location);
CREATE INDEX idx_well_status ON wells(status);
CREATE INDEX idx_well_type ON wells(type);
"""
run_query(query)


CREATE INDEX


In [11]:
import db_settings
import psycopg2

conn = psycopg2.connect(database=db_settings.DB, user=db_settings.USER, password=db_settings.PASSWD, host=db_settings.HOST)
with conn:
    with conn.cursor() as cur:
        with open('states-data.csv', 'r') as f:
            cur.copy_expert('COPY states(api_prefix, state, name) FROM STDIN WITH DELIMITER \',\' CSV HEADER', f)
            #, 'states', sep=',', columns='api_prefix, state, name'))
conn.close()            